## Installing libraries

In [84]:
# Importing the required libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import datetime

In [85]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
db_name = 'Banks.db'
csv_name = 'Largest_banks_data.csv'
table_name = 'Largest_banks'

csv_path = f'output/{csv_name}'
db_path = f'{db_name}'

log_file = "code_log.txt"
log_path = f"log/{log_file}"


## Logging function:
- function to log the progress of the code.
- The function accepts the message to be logged and enters it to a text file code_log.txt.

In [86]:
def log_progress(message): 
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    log_file_path = log_path
    
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    
    with open(log_file_path,"a") as f: 
        f.write(timestamp + ':' + message + '\n')

## Extraction of data

In [87]:
from io import StringIO


def extract(url, table_attribs):
    """ This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. """

    soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    table = soup.find('span', string=table_attribs).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]

    log_progress('Data extraction complete. Initiating Transformation process')

    return df

In [88]:
df = extract(url , "By market capitalization")

## Transformation of data:
- Read the exchange rate CSV file.
- Converting the contents to a dictionary.
- Adding 3 different columns to the dataframe, { MC_GBP_Billion, MC_EUR_Billion and MC_INR_Billion} containing the content of MC_USD_Billion scaled by the corresponding exchange rate factor.
- Rounding the resulting data to 2 decimal places.

In [89]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()['Rate']

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['GBP'], 2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)'] * exchange_rate['INR'], 2)
    df = df.rename(columns={"Bank name": "Name"})

    log_progress('Data transformation complete. Initiating Loading process')

    return df

In [90]:
df2 = transform(df , "exchange_rate.csv")

df2

,Rank,Name,Market cap (US$ billion),MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,1,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,2,Bank of America,231.52,185.22,215.31,19204.58
2,3,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,4,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,5,HDFC Bank,157.91,126.33,146.86,13098.63
5,6,Wells Fargo,155.87,124.70,144.96,12929.42
6,7,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
7,8,Morgan Stanley,140.83,112.66,130.97,11681.85
8,9,China Construction Bank,139.82,111.86,130.03,11598.07
9,10,Bank of China,136.81,109.45,127.23,11348.39


## Loading Dataframe to CSV File:

In [91]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

    df.to_csv(output_path)

    log_progress('Data saved to CSV file')

In [92]:
load_to_csv(df2 , "output/Largest_banks.csv")

## Loading Dataframe to database:


In [93]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

    log_progress('Data loaded to Database as a table, Executing queries')

In [94]:
# Establish a connection to the SQLite3 database
import sqlite3

connection = sqlite3.connect('Banks.db')
load_to_db(df2, connection , "Largest_banks")

## Run queries on Database

In [95]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''

    df = pd.read_sql_query(query_statement , sql_connection)
    print(df)
   
    log_progress('Process Complete')

In [96]:
run_query('SELECT * FROM Largest_banks', connection)
run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', connection)
run_query('SELECT Name from Largest_banks LIMIT 5', connection)

   Rank                                     Name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2                          Bank of America                    231.52   
2     3  Industrial and Commercial Bank of China                    194.56   
3     4               Agricultural Bank of China                    160.68   
4     5                                HDFC Bank                    157.91   
5     6                              Wells Fargo                    155.87   
6     7                        HSBC Holdings PLC                    148.90   
7     8                           Morgan Stanley                    140.83   
8     9                  China Construction Bank                    139.82   
9    10                            Bank of China                    136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215.31    